In [1]:
#Import TensorBoard For Logging
%load_ext tensorboard 

In [2]:
import h5py
import os
import tensorflow as tf
import glob
import datetime
import gc
from keras.callbacks import ModelCheckpoint
# import keras
import pandas as pd
import numpy as np
import cv2
from scipy.signal import medfilt
import matplotlib.pyplot as plt
import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline

# Data Reading / Pre-Processing (Unlabeled Data)


## DATA Import Code (Via Drive)

In [3]:
#Mount Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Function to Read Unlabelled Data

In [4]:
CTW_unlabelled = "/content/drive/MyDrive/CTW2020/Unzipped/CTW2020_unlabelled_data/"
# path might vary based on how you saved data in your drive

In [5]:
def get_data(data_file):
    
    f = h5py.File(data_file, 'r')
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR = f['SNR'][:] #shape (sample size, 56, 5)
    f.close()
            
    return H_Re, H_Im, SNR

In [6]:
#lists all the files

for dirname, _, filenames in os.walk(CTW_unlabelled):
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))
    print("Total number of files = {}".format(len(filenames)))


Total number of files = 64


In [7]:
def preprocessing(H_Re, H_Im):
  amp = tf.math.sqrt(tf.math.square(H_Re) + tf.math.square(H_Im))
  amp = medfilt(amp, [1, 1, 1, 3])
  normalized_amp = tf.keras.utils.normalize(amp)
  # avg_amp = tf.reduce_sum(amp, [2, 3])/(amp.shape[2] * amp.shape[3])
  # max_amp = tf.reduce_max(amp, 3)
  # min_amp = tf.reduce_min(amp, 3)
  # if (max_amp - min_amp !=0):
  #   normalized_amp = (tf.reshape(amp, [amp.shape[3], amp.shape[0], amp.shape[1], amp.shape[2]]) - min_amp) / (max_amp - min_amp)
  #   mormalized_amp = tf.reshape(normalized_amp, [amp.shape[0], amp.shape[1], amp.shape[2], amp.shape[3]])
  #   normalized_amp = tf.reshape(normalized_amp, [amp.shape[2], amp.shape[3], amp.shape[0], amp.shape[1]]) * (avg_amp / tf.reduce_max(max_amp))
  #   normalized_amp = tf.reshape(normalized_amp, [amp.shape[0], amp.shape[1], amp.shape[2], amp.shape[3]])
  # else:
  #     normalized_amp = amp
  return normalized_amp

In [8]:
## Calling with Start as 1,2,3 will load in distinct sets of 10 (Or mentioned) files 

def LoadData1():
  H_Re=[]
  H_Im=[]
  SNR=[]
  #print("Loading Files")
  while True:
    for i in range(1):
      filename = CTW_unlabelled + "file_{}.hdf5".format(i+1)
      H_Re, H_Im, SNR= get_data(filename)
      i+=1
      # YOUR_CODE
      # amplitude = preprocessing(H_Re, H_Im)
      # filtered_amp = 
      normalized_amp = preprocessing(H_Re, H_Im)

      filtered_snr = medfilt(SNR, [1, 1, 3])
      # normalized_snr =  
     
      yield (normalized_amp,filtered_snr)

In [9]:
## Calling with Start as 1,2,3 will load in distinct sets of 10 (Or mentioned) files 

def LoadData2():
  H_Re=[]
  H_Im=[]
  SNR=[]
  #print("Loading Files")
  while True:
    for i in range(64):
      filename = CTW_unlabelled + "file_{}.hdf5".format(i+1)
      H_Re, H_Im, SNR= get_data(filename)
      i+=1
       # YOUR_CODE
      # amplitude = 
      # filtered_amp = 
      normalized_amp = preprocessing(H_Re, H_Im)

      filtered_snr = medfilt(SNR, [1, 1, 3])
      # normalized_snr =  
     
      yield (filtered_snr,normalized_amp) #Note the difference in order of variables compared to loaddata1 function

     

# Define and Train Pretext Task Models

In [10]:
tf.keras.backend.clear_session()

## Model 1 (Uses H_Re and H_Im)

In [11]:
# Clear any logs from previous runs (Tensorboard)
!rm -rf ./logs/

In [12]:
##Tried Leaky Relu -  Errors are too high , so used tanh instead

#Base Model is the part we're using for transfer learning
#i.e The part that will be transfered

#YOUR_CODE
#Add CNNs to the basemodel
BaseModel1 = tf.keras.Sequential([
          tf.keras.layers.Conv2D(128, (3, 3), activation='tanh'),
          tf.keras.layers.MaxPooling2D((5,5)),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Conv2D(64, (3, 3), activation='tanh'),
          tf.keras.layers.MaxPooling2D((3, 3)),
          tf.keras.layers.Dropout(0.3),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(462),
          tf.keras.layers.Activation('tanh'),
          tf.keras.layers.Dense(231)])

model1 = tf.keras.Sequential([tf.keras.Input(shape=(56,924,5)),BaseModel1])
model1.add(tf.keras.layers.Activation('tanh'))
# model1.add(tf.keras.layers.Dense(112))
# model1.add(tf.keras.layers.Activation('tanh'))
model1.add(tf.keras.layers.Dense(280))
model1.add(tf.keras.layers.Reshape((56,5)))
print(model1.output_shape)

(None, 56, 5)


In [13]:
print(BaseModel1.summary())
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 922, 128)      5888      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 184, 128)      0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 184, 128)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 182, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 60, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 60, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 7680)              0

In [14]:
model1.compile(optimizer='adam', loss='mse',metrics=['accuracy','mean_absolute_percentage_error'])

## Train Pretext Model - Part 1 (Using Unlabelled data)

In [15]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
savefilepath1 = "/content/drive/MyDrive/CTW2020/Checkpoints/PretextModel1/best_model.hdf5"
checkpoint = ModelCheckpoint(savefilepath1, monitor='loss', verbose=1,save_best_only=True, mode='auto')

In [ ]:
gc.collect()       #Was running out of ram - Added to see if it helps
model1.fit(LoadData1(),epochs=5,steps_per_epoch=64 ,verbose = 1,callbacks=[tensorboard_callback,checkpoint])
model1.save_weights(savefilepath, overwrite=True, save_format=None, options=None)

Epoch 1/5


In [ ]:
model1.load_weights(savefilepath1, by_name=False, skip_mismatch=False, options=None)

In [ ]:
%tensorboard --logdir logs/fit

## Model 2 (Uses SNR)

In [ ]:
# Clear any logs from previous run (Tensorboard) for the next train sequence (!!!CHECK FOR MORE EFFICIENT WAY)
!rm -rf ./logs/

In [ ]:
#YOUR_CODE
#Add CNNs to the basemodel
BaseModel2 = tf.keras.Sequential([tf.keras.layers.Dense(112),tf.keras.layers.Activation('tanh'),tf.keras.layers.Dense(231)])

model2 = tf.keras.Sequential([tf.keras.Input(shape=(56,5)),BaseModel2])
model2.add(tf.keras.layers.Activation('tanh'))
model2.add(tf.keras.layers.Dense(924))
model2.add(tf.keras.layers.Activation('tanh'))
model2.add(tf.keras.layers.Dense(103488))
model2.add(tf.keras.layers.Reshape((2,56,924)))
print(model2.output_shape)

(None, 2, 56, 924)


In [ ]:
print(BaseModel2.summary())
print(model2.summary())

In [ ]:
model2.compile(optimizer='sgd', loss='mse',metrics=['accuracy','mean_absolute_percentage_error'])

## Train Pretext Model - Part 2 (Using Unlabelled data)

In [ ]:
savefilepath2 = "/content/drive/MyDrive/Datasets/CTW2020/Checkpoints/PretextModel2/best_model.hdf5"
checkpoint = ModelCheckpoint(savefilepath2, monitor='loss', verbose=1,save_best_only=True, mode='auto')

In [ ]:
gc.collect()       #Was running out of ram - Added to see if it helps
model2.fit(LoadData2(),epochs=5,steps_per_epoch=64 ,verbose = 1,callbacks=[tensorboard_callback,checkpoint])
model2.save_weights(savefilepath, overwrite=True, save_format=None, options=None)

In [ ]:
model1.load_weights(savefilepath2, by_name=False, skip_mismatch=False, options=None)

In [ ]:
%tensorboard --logdir logs/fit

# Read Labelled data - For downstream Training

In [ ]:
# Clear any logs from previous run (Tensorboard) for the next train sequence (!!!CHECK FOR MORE EFFICIENT WAY)
!rm -rf ./logs/

In [ ]:
CTW_labelled = "/content/drive/My Drive/Datasets/CTW2020/Unzipped/CTW2020_labelled_data/"
## path might vary based on how you saved data in your drive

In [ ]:
def get_data(data_file):
    
    f = h5py.File(data_file, 'r')
    H_Re = f['H_Re'][:] #shape (sample size, 56, 924, 5)
    H_Im = f['H_Im'][:] #shape (sample size, 56, 924, 5)
    SNR = f['SNR'][:] #shape (sample size, 56, 5)
    Pos = f['Pos'][:] #shape(sample size, 3)
    f.close()
            
    return H_Re, H_Im, SNR, Pos        

In [ ]:
#lists all the files

for dirname, _, filenames in os.walk(CTW_labelled):
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))
    print("Total number of files = {}".format(len(filenames)))

Total number of files = 9


In [ ]:
## Calling with Start as 1,2,3 will load in distinct sets of 10 (Or mentioned) files 

def LoadData2():
  H_Re=[]
  H_Im=[]
  SNR=[]
  #print("Loading Files")
  for i in range(64):
    filename = CTW_unlabelled + "file_{}.hdf5".format(i+1)
    H_Re, H_Im, SNR= get_data(filename)
    i+=1
    
    #YOUR_CODE

    #Do the data preprocessing as before

    yield (SNR , AMP)

In [ ]:
def LoadData_FinalModel():

  H_Re=[]
  H_Im=[]
  SNR=[]
  Pos=[]
  
  while True:
    #print("Loading Files")
    for i in range(9):
      filename = CTW_labelled + "file_{}.hdf5".format(i+1)
      H_Re, H_Im, SNR ,Pos= get_data(filename)
      i+=1
  
  #YOUR_CODE

    #Do the data preprocessing as before
    
  
      yield ((AMP,SNR),Pos)

# Downstream Model 

In [ ]:
BaseModel1.trainable = False
BaseModel2.trainable = False

In [ ]:
Input1 = tf.keras.Input(shape=(2,56,924))
Input2 = tf.keras.Input(shape=(56))

Output = tf.keras.layers.concatenate([BaseModel1(Input1), BaseModel2(Input2)], axis = 1)

Output = tf.keras.layers.Activation('tanh')(Output)
Output = tf.keras.layers.Dense(100)(Output)
Output = tf.keras.layers.Activation('tanh')(Output)
Output = tf.keras.layers.Dense(56)(Output)
Output = tf.keras.layers.Activation('tanh')(Output)
Output = tf.keras.layers.Dense(3)(Output)

FinalModel = tf.keras.Model(inputs=[Input1,Input2], outputs=Output)

In [ ]:
FinalModel.summary()

In [ ]:
FinalModel.compile(optimizer='sgd', loss='mse',metrics=['accuracy','mean_absolute_percentage_error'])

In [ ]:
savefilepath_FinalModel = "/content/drive/My Drive/Datasets/CTW2020/Checkpoints/DownstreamModel/best_model.hdf5"
checkpoint = ModelCheckpoint(savefilepath_FinalModel, monitor='loss', verbose=1,save_best_only=True, mode='auto')

In [ ]:
gc.collect()       #Was running out of ram - Added to see if it helps
FinalModel.fit(LoadData_FinalModel(),steps_per_epoch=9 ,epochs=5 ,verbose = 1,callbacks=[tensorboard_callback,checkpoint])
FinalModel.save_weights(savefilepath_FinalModel, overwrite=True, save_format=None, options=None)

In [ ]:
FinalModel.load_weights(savefilepath, by_name=False, skip_mismatch=False, options=None)

In [ ]:
%tensorboard --logdir logs/fit

##Code for Tuning Part

In [ ]:
# Clear any logs from previous run (Tensorboard) for the next train sequence (!!!CHECK FOR MORE EFFICIENT WAY)
# !rm -rf ./logs/

#Make trainable
BaseModel1.trainable = True
BaseModel2.trainable = True

In [ ]:
FinalModel.compile(optimizer='sgd', loss='mse',metrics=['accuracy','mean_absolute_percentage_error'])

In [ ]:
gc.collect()       #Was running out of ram - Added to see if it helps
FinalModel.fit(LoadData_FinalModel(),epochs=5 , steps_per_epoch=9 ,verbose = 1,callbacks=[tensorboard_callback,checkpoint])
FinalModel.save_weights(savefilepath_FinalModel, overwrite=True, save_format=None, options=None)

In [ ]:
FinalModel.load_weights(savefilepath_FinalModel, by_name=False, skip_mismatch=False, options=None)